In [1]:
import sys
sys.path.insert(0, "../src")

In [2]:
import re
import yaml
from sariModelInterpreter import modelInterpreter as mi

In [3]:
inputFile = '../models/bso.yml'

In [4]:
model = mi.parseModelFromFile(inputFile)

In [5]:
mi.verifyModel(model)

'Ok'

In [6]:
print(mi.compileQueryForNodes(model, 
                              'artwork',
                              ['work_creator', 'artwork_collection', 'production_date_formatted', 'work_creation_date_formatted'], 
                              group=['work_creator_label'],
                              optional=['production_date_formatted', 'work_creation_date_formatted'],
                              inject=[{
                                  'id': 'work_creator_label',
                                  'query': '$value_work_creator rdfs:label ?value .'
                              },{
                                  'id': 'artwork_label',
                                  'query': '$subject rdfs:label ?value .'
                              }],
                              unselect=['work_creator']))


SELECT ($subject as ?artwork) (?value_artwork_collection as ?artwork_collection) (?value_production_date_formatted as ?production_date_formatted) (?value_work_creation_date_formatted as ?work_creation_date_formatted) (GROUP_CONCAT(?value_work_creator_label) as ?work_creator_label) (?value_artwork_label as ?artwork_label) {
$value_work_creator rdfs:label ?value_work_creator_label .
$subject rdfs:label ?value_artwork_label .
 $subject crm:P128_carries ?value_work . 
 ?value_work crm:P94i_was_created_by ?value_work_creation . 
 ?value_work_creation crm:P14_carried_out_by ?value_work_creator . 
 $subject crm:P109_has_former_or_current_curator ?value_artwork_collection . 
OPTIONAL {
	 $subject crm:P108i_was_produced_by ?value_production . 
	 ?value_production crm:P4_has_time-span ?value_production_date . 
	 ?value_production_date crm:P82a_begin_of_the_begin ?from_production_date_formatted ; crm:P82b_end_of_the_end ?to_production_date_formatted . BIND(IF(YEAR(?from_production_date_formatted)